## MAIN Jupyter Notebook
Modified from: https://www.kaggle.com/code/saekiryosuke/fine-tuned-classifier-with-vgg16-and-pytorch

In [1]:
# import packages
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import models, transforms
import pandas as pd


In [2]:
class ImageTransform():

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # data augmentation
                transforms.RandomResizedCrop(
                   resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(), 
                # convert to tensor for PyTorch
                transforms.ToTensor(),
                # color normalization
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase='train'):

        return self.data_transform[phase](img)

In [3]:
import os
import glob

# Get the current directory
current_directory = "./data/kaggle/collected/non_violent"

# Use glob to find all .jpg files in the current directory
collected_nv = glob.glob(os.path.join(current_directory, '*'))
current_directory2 = "./data/kaggle/collected/violent"

# Use glob to find all .jpg files in the current directory
collected_v = glob.glob(os.path.join(current_directory2, '*'))

# jpg_files now contains a list of all .jpg files in the current directory
print('size of non-violent: ', len(collected_nv))
print('size of violent: ', len(collected_v))


size of non-violent:  708
size of violent:  98


In [4]:
# def remove_alpha_channel(image_path, output_path):
#     # Open the image using PIL
#     img = Image.open(image_path)

#     # Ensure the image has an alpha channel
#     if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
#         # Convert to RGB mode
#         img = img.convert('RGB')

#     # Save the image without the alpha channel
#     img.save(output_path)

# for i in collected_nv:
#     remove_alpha_channel(i, i)

# for i2 in collected_v:
#     remove_alpha_channel(i2, i2)

In [5]:
# import os
# import glob

# # Get the current directory
# current_directory = "./data/kaggle/collected/non_violent"

# # Use glob to find all .jpg files in the current directory
# collected_nv = glob.glob(os.path.join(current_directory, '*'))
# current_directory2 = "./data/kaggle/collected/violent"

# # Use glob to find all .jpg files in the current directory
# collected_v = glob.glob(os.path.join(current_directory2, '*'))

# # jpg_files now contains a list of all .jpg files in the current directory
# print('size of non-violent: ', len(collected_nv))
# print('size of violent: ', len(collected_v))


In [6]:
def make_datapath_list(rootpath):
    
    dir_list = os.listdir(rootpath)

    return dir_list


violent_list = make_datapath_list("./data/kaggle/Violent")
notviolent_list = make_datapath_list("./data/kaggle/Non-Violent")

In [7]:
collected_img_train = []
for f in collected_nv[:68]:
	collected_img_train.append((Image.open(f), 0))

for f in collected_v[:68]:
	collected_img_train.append((Image.open(f), 1))

collected_img_val	= []
for f in collected_nv[68:77]:
	collected_img_val.append((Image.open(f), 0))

for f in collected_v[68:77]:
	collected_img_val.append((Image.open(f), 1))

collected_img_test	= []
for f in collected_nv[77:]:
	collected_img_test.append((Image.open(f), 0))

for f in collected_v[77:]:
	collected_img_test.append((Image.open(f), 1))

In [8]:

# images_train = []
# for f in violent_list[:4000]:
#     images_train.append((Image.open(f'./data/kaggle/Violent/{f}'), 1))

# for f in notviolent_list[:4000]:
#     images_train.append((Image.open(f'./data/kaggle/Non-Violent/{f}'), 0))

# images_val = []
# for f in violent_list[4000:4500]:
#     images_val.append((Image.open(f'./data/kaggle/Violent/{f}'), 1))

# for f in notviolent_list[4000:4500]:
#     images_val.append((Image.open(f'./data/kaggle/Non-Violent/{f}'), 0))

# images_test = []
# for f in violent_list[4500:5000]:
#     images_test.append((Image.open(f'./data/kaggle/Violent/{f}'), 1))

# for f in notviolent_list[4500:5000]:
#     images_test.append((Image.open(f'./data/kaggle/Non-Violent/{f}'), 0))

# print(len(images_train))
# print(len(images_val))
# print(len(images_test))


In [9]:
def pil_images_to_numpy(images):
    image_data = [np.array(image[0].resize((128,128)))[:,:,:3] for image in images]
    return image_data

In [10]:
import random

# image_data = pil_images_to_numpy(random.sample(images_train, 1000))
image_data = pil_images_to_numpy(collected_img_train)

# # Calculate the mean and standard deviation
# mean = np.mean(image_data, axis=(0, 1, 2))
# stddev = np.std(image_data, axis=(0, 1, 2))
mean = np.mean(image_data, axis=(0, 1, 2))
stddev = np.std(image_data, axis=(0, 1, 2))
# print('online dataset stats: ', mean, stddev)
print('collected dataset stats: ', mean, stddev)

collected dataset stats:  [129.58357418 123.72287616 107.8359366 ] [76.57440544 72.6437862  71.48943184]


In [11]:
# mean = (0.5, 0.5, 0.5)
# stddev = (0.5, 0.5, 0.5)

In [12]:
class imageDataset(data.Dataset):

    def __init__(self, img_list, transform=None, phase='train'):
        self.img_list = img_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):

        # load image
        img = self.img_list[index][0]
        label = self.img_list[index][1]

        # resize
        img = img.resize((256, 256))
        
        # # # grey -> color
        # img = img.convert("L").convert("RGB")

        # preprocess
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        return img_transformed, label

# run
train_dataset = imageDataset(
    img_list=collected_img_train, transform=ImageTransform(256, mean, stddev), phase='train')

val_dataset = imageDataset(
    img_list=collected_img_val, transform=ImageTransform(256, mean, stddev), phase='val')



In [13]:
batch_size = 32

# making dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# put dataloader into dictionary type
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


In [18]:
# Initialize the model
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
# setting of loss function
criterion = nn.CrossEntropyLoss()

# setting fine tuned parameters
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# Not only output layer, "features" layers and other classifier layers are tuned.
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# store parameters in list
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        #print("params_to_update_1:", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        #print("params_to_update_2:", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        #print("params_to_update_3:", name)

    else:
        param.requires_grad = False
        #print("no learning", name)

# print("-----------")
# print(params_to_update_1)

# Learning Rates
optimizer = optim.Adam([
    {'params': params_to_update_1, 'lr': 1e-5},
    {'params': params_to_update_2, 'lr': 1e-5},
    {'params': params_to_update_3, 'lr': 1e-5}
])

In [15]:
# training function
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    accuracy_list = []
    loss_list = []
    
    # Precondition : Accelerator GPU -> 'On'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using device：", device)

    # put betwork into GPU
    net.to(device)
    torch.backends.cudnn.benchmark = True

    # epoch loop
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # set network 'train' mode
            else:
                net.eval()   # set network 'val' mode

            epoch_loss = 0.0
            epoch_corrects = 0

            # Before training
            if (epoch == 0) and (phase == 'train'):
                continue
            
                      
            # batch loop
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                   
                # send data to GPU
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # initialize optimizer
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  #calcurate loss
                    _, preds = torch.max(outputs, 1)  # predict
  
                    # back propagtion
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # update loss summation
                    epoch_loss += loss.item() * inputs.size(0)  
                    # update correct prediction summation
                    epoch_corrects += torch.sum(preds == labels.data)

            # loss and accuracy for each epoch loop
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val':
                accuracy_list.append(epoch_acc.item())
                loss_list.append(epoch_loss)
            
    return accuracy_list, loss_list

In [19]:
num_epochs=20
accuracy_list, loss_list = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

using device： cuda:0
Epoch 1/20
-------------


100%|██████████| 1/1 [00:00<00:00,  3.27it/s]


val Loss: 0.6940 Acc: 0.5000
Epoch 2/20
-------------


100%|██████████| 5/5 [00:03<00:00,  1.42it/s]


train Loss: 0.7106 Acc: 0.4338


...

val Loss: 0.4828 Acc: 1.0000
Epoch 20/20
-------------


100%|██████████| 5/5 [00:03<00:00,  1.58it/s]


train Loss: 0.6043 Acc: 0.6691


100%|██████████| 1/1 [00:00<00:00,  3.98it/s]

val Loss: 0.4507 Acc: 0.9444


In [20]:
torch.save(net.state_dict(), './model-weights/vgg16_fine_tuning-5e.pt')